In [1]:
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2
from IPython.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
import random
from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import stox
# from ib_insync import *
from stox.imports import *
from stox.ibkr import IB_Extractor, describe_contract, download_reports, IBKR_Loader, IBKR_Stats, IBKR_Derived
from stox.yahoo import Yahoo_Loader, Yahoo_Info
from tqdm.notebook import tqdm
import plotly.io as pio
import plotly.express as px
from omnibelt import tqdmd_notebook as tqdmd
from stox import misc, yahoo, ibkr
from stox.general import Quantity, PctChange, PopulationStats, country_colors, sector_symbols
from collections import namedtuple
import pandas as pd
# import qgrid
from omniply import tool, ToolKit, Context, Scope, Selection
sns.set_theme(style="darkgrid")
pio.renderers.default = "browser"

In [2]:
ibroot = misc.assets_root() / 'ibkr'
path = ibroot / 'fe-aug23.txt'
# path = ibroot / 'gu-aug23.txt'
portfolio_list = misc.extract_tickers_and_shares(path)
current_portfolio = {k: v for k, v in portfolio_list}
assert len(current_portfolio) == len(portfolio_list)
symbol_table = stox.load_symbol_table()
len(symbol_table), len(current_portfolio)

(274, 58)

In [3]:
rows = [(k, v) for k, v in symbol_table.items() if v['currency'] == 'EUR']
def from_yfsym(yfsym):
	return yfsym, [v for k, v in rows if k == yfsym][0]
def from_ibsym(ibsym):
	return [(k,v) for k, v in rows if v['symbol'] == ibsym][0]
yf2ib = {k: v for k, v in rows}
yfsyms = [k for k, v in rows]
len(rows), len(yf2ib)

(241, 241)

In [4]:
date = '230829'
date = '230908'
# date = 'last'

def get_context(yfsym):
	ibrow = yf2ib[yfsym]
	
	gg = Context(
		# Selection(IBKR_Derived(), IBKR_Stats(), IBKR_Loader(root=misc.ibkr_root()),
		# 	  gap={
		# 		  'ckpt_path': 'ib_ckpt_path',
		# 	  	
		# 	  		'snapshot': 'snapshot',
		# 	  
		# 	  }),
		Scope(Yahoo_Info(), Yahoo_Loader(root=misc.yahoo_root()),
			  gap={
				  'ckpt_path': 'yahoo_ckpt_path',
				  'ticker': 'yfsym',
				  
				  # 'info': 'info',
					#   'splits': 'splits',
				  # 
				  # 'recommendation_mean': 'recommendation_mean',
				  # 'recommendation_key': 'recommendation_key',
				  # 'number_of_analysts': 'number_of_analysts',
				  # 'target_mean_price': 'target_mean_price',
				  # 'target_high_price': 'target_high_price',
				  # 'target_low_price': 'target_low_price',
				  # 'target_median_price': 'target_median_price',
				  # 
				  # 'isin': 'yf_isin',
				  # 
				  # 'market_cap': 'yf_market_cap',
				  # 'sector': 'yf_sector',
			  }),
	)
	gg.update(ibrow)
	gg['date'] = date
	gg['ibsym'] = ibrow['symbol']
	gg['ibid'] = ibrow['conId']
	gg['yfsym'] = yfsym
	return gg

In [5]:
ctxs = [get_context(yfsym) for yfsym in yfsyms]
len(ctxs)

241

In [6]:
# print([ctx['yfsym'] for ctx in bad]) # snapshot ['APAM.BR', 'ARGX.BR', 'BELA.AT', 'CABK.MC', 'EBS.VI', 'HTO.AT', 'KER.PA', 'LR.PA', 'OR.PA', 'PRY.MI', 'RACE.MI', 'SAR.AT', 'STR.VI', 'UMI.BR', 'ZAL.DE']
# print([ctx['yfsym'] for ctx in bad]) # recommendations ['APAM.BR', 'ARGX.BR', 'BELA.AT', 'CABK.MC', 'EBS.VI', 'ELLAKTOR.AT', 'EYDAP.AT', 'HTO.AT', 'KER.PA', 'LR.PA', 'OR.PA', 'PRY.MI', 'RACE.MI', 'SAR.AT', 'STR.VI', 'UMI.BR', 'VIO.AT', 'ZAL.DE']
# print([ctx['yfsym'] for ctx in bad]) # info ['APAM.BR', 'CON.DE']

In [7]:
pop = []
bad = []
for ctx in tqdmd(ctxs, key=lambda x: x['yfsym']):
	try:
		# ctx['snapshot']
		# ctx['recommendations']
		ctx['info']
		assert ctx['sector'] is not None
	except Exception as e:
		raise e
		bad.append(ctx)
	else:
		pop.append(ctx)
print([ctx['yfsym'] for ctx in bad])
ctx = pop[0]
len(pop), len(bad)

  0%|          | 0/241 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\anwan\\OneDrive\\Khan\\projects\\stox\\stoxdata\\yahoo_data\\APAM_BR\\230908\\info.json'

In [ ]:
Counter([ctx['sector'] for ctx in pop])

In [ ]:
Counter([ctx['country'] for ctx in pop])

In [10]:
# Counter([ctx['industry'] for ctx in pop])

In [11]:
info = ctx['info']

In [12]:
# key1, key2 = 'peg_ratio', 'beta'
# def selector(ctx):
# 	try:
# 		val = ctx[key1]
# 		assert (val.amount if isinstance(val, Quantity) else val) is not None
# 		val = ctx[key2]
# 		assert (val.amount if isinstance(val, Quantity) else val) is not None
# 		val = ctx['market_cap']
# 		assert (val.amount if isinstance(val, Quantity) else val) is not None
# 	except:
# 		return False
# 	return True
# pts = [c for c in pop if selector(c)]
# print(len(pts), len(pop))
# x = np.array([p[key1].amount if isinstance(p[key1], Quantity) else p[key1] for p in pts])
# y = np.array([p[key2].amount if isinstance(p[key2], Quantity) else p[key2] for p in pts])
# sizes = np.array([np.log10(ctx['market_cap'].amount) for ctx in pts])
# sizes = 10 + 30 * (sizes - sizes.min()) / (sizes.max() - sizes.min())
# r = np.corrcoef(x, y)[0, 1]
# fg = px.scatter(x=x, y=y, 
# 				hover_name=[f'{c["yfsym"]}  ::  {c["company_short_name"]}' for c in pts], 
#     size=sizes, 
#     color=[c['country'] for c in pts],  # differentiate markers by color
#     color_discrete_map=country_colors,  # Map colors
#     symbol=[c['sector'] for c in pts],  # differentiate markers by symbol
#     symbol_map=sector_symbols,  # Map shapes
# 				# trendline='ols'
# 				title=f'Correlation = {r:.2f}',
# 				log_x=True,
# 				# log_y=True,
# 				)
# fg.update_layout(xaxis_title=key1)
# fg.update_layout(yaxis_title=key2)
# fg.show();
# # sns.regplot(x=x, y=y)
# # plt.title(f'Correlation = {r:.2f}');

In [13]:
# from omniply import AbstractGadget
# 
# class PopulationStats(AbstractGadget):
# 	def __init__(self, population: list, *gizmos: str, percentile=False, location=True):
# 		super().__init__()
# 		self._population = population
# 		self._gizmos = gizmos
# 		self._location = location
# 		self._percentile = percentile
# 
# 	def gizmos(self) -> Iterator[str]:
# 		if self._location:
# 			yield from (f'loc_{gizmo}' for gizmo in self._gizmos)
# 		if self._percentile:
# 			yield from (f'pct_{gizmo}' for gizmo in self._gizmos)
# 
# 
# 	def _base_stats(self, pop, key):
# 		for item in pop:
# 			val = item[key]
# 			if val is not None and (not isinstance(val, Quantity) or val.amount is not None):
# 				yield val
# 
# 
# 	def compute_pct(self, mark, key: str) -> float:
# 		count = []
# 		for val in self._base_stats(self._population, key):
# 			if val is not None:
# 				if val == mark:
# 					count.append(0.5)
# 				elif val < mark:
# 					count.append(1)
# 				else:
# 					count.append(0)
# 		assert len(count) > 0, f'No values for {key}'
# 		return int(100 * sum(count) / len(count))
# 
# 	def compute_loc(self, mark, key: str) -> str:
# 		count = []
# 		for val in self._base_stats(self._population, key):
# 			if val is not None:
# 				if val == mark:
# 					count.append(0.5)
# 				elif val < mark:
# 					count.append(1)
# 				else:
# 					count.append(0)
# 		assert len(count) > 0, f'No values for {key}'
# 		return f'{int(sum(count))}/{len(count)}'
# 
# 	def grab_from(self, ctx, gizmo: str):
# 		if self._percentile and gizmo.startswith('pct_'):
# 			key = gizmo[4:]
# 			mark = ctx[key]
# 			return self.compute_pct(mark, key)
# 		if self._location and gizmo.startswith('loc_'):
# 			key = gizmo[4:]
# 			mark = ctx[key]
# 			return self.compute_loc(mark, key)
# 		raise NotImplementedError

In [14]:
pop_sec = {}
for ctx in pop:
	pop_sec.setdefault(ctx['sector'], []).append(ctx)
print({k:len(v) for k, v in pop_sec.items()})
to_group = {
	'Portugal': 'Other', 
	'Luxembourg': 'Other', 
	'Sweden': 'Other',
	'Switzerland': 'Other',
}
pop_land = {}
for ctx in pop:
	pop_land.setdefault(to_group.get(ctx['country'],ctx['country']), []).append(ctx)
print({k:len(v) for k, v in pop_land.items()})

{'Consumer Defensive': 15, 'Basic Materials': 18, 'Financial Services': 43, 'Consumer Cyclical': 24, 'Industrials': 47, 'Technology': 15, 'Real Estate': 11, 'Healthcare': 14, 'Utilities': 20, 'Communication Services': 13, 'Energy': 10}
{'Other': 6, 'Germany': 30, 'Belgium': 21, 'Netherlands': 20, 'France': 32, 'Spain': 28, 'Greece': 20, 'Italy': 29, 'Austria': 20, 'Finland': 19, 'United Kingdom': 5}


In [15]:
stat_names = [
	'peg_ratio',
	'market_cap',
	'beta',
	'change_52w',
	'yield',
	'perf52w',
	'recommendation_mean',
	'overall_risk',
	'held_percent_institutions',
	'held_percent_insiders',
]
basic_names = [
	'employees',
]
cat_names = [
	'country',
	'sector',
	'industry',
	'recommendation_key',
]
text_names = [
	'company_name',
	'company_short_name',
	# 'city',
]
feature_names = [*stat_names, *basic_names, *cat_names, *text_names]

In [16]:
pop_stat_type = 'pct'
global_pop_stats = Scope(
	PopulationStats(pop, *stat_names, percentile=pop_stat_type == 'pct', location=pop_stat_type == 'loc'), 
	gap={f'{pop_stat_type}_{key}': f'glob_{pop_stat_type}_{key}' for key in stat_names}
)
sec_pop_stats = {sec: Scope(
	PopulationStats(pop_sec[sec], *stat_names, percentile=pop_stat_type == 'pct', location=pop_stat_type == 'loc'), 
	gap={f'{pop_stat_type}_{key}': f'sec_{pop_stat_type}_{key}' for key in stat_names}
) for sec, ctxs in pop_sec.items()}

from stox.general import country_flags, sector_emojis

display_countries = {k: f'{e} {k}' for k, e in country_flags.items()}
display_sectors = {k: f'{e} {k}' for k, e in sector_emojis.items()}

display_country = tool('display_country')(lambda country: display_countries.get(country, country))
display_sector = tool('display_sector')(lambda sector: display_sectors.get(sector, sector))

share_counter = tool('shares')(lambda ibsym: current_portfolio.get(ibsym, 0))
market_cap_raw_tool = tool('market_cap_raw')(lambda market_cap: market_cap.amount)

for ctx in pop:
	ctx.include(market_cap_raw_tool, share_counter, display_sector, display_country, global_pop_stats, sec_pop_stats[ctx['sector']])
len(sec_pop_stats)

11

In [17]:
ctx

Context(conId, currency, exchange, primaryExchange, symbol, date, ibsym, ibid, yfsym, info, sector, country, yahoo_ckpt_path, {city}, {website}, {industry}, {business_summary}, {employees}, {audit_risk}, {board_risk}, {compensation_risk}, {share_holder_risk}, {overall_risk}, {high_52w}, {low_52w}, {cash_per_share}, {debt_to_equity}, {earnings_growth}, {yield}, {revenue}, {gross_profit}, {free_cash_flow}, {operating_cash_flow}, {operating_income}, {ebitda}, {debt}, {cash}, {current_ratio}, {quick_ratio}, {beta}, {trailing_pe}, {forward_pe}, {volume}, {volume_10d}, {market_cap}, {peg_ratio}, {price_to_book}, {trailing_eps}, {forward_eps}, {company_short_name}, {company_name}, {price}, {recommendation_mean}, {recommendation_key}, {number_of_analysts}, {target_mean_price}, {target_high_price}, {target_low_price}, {target_median_price}, {change_52w}, {held_percent_institutions}, {held_percent_insiders}, {profit_margins}, {target_mean_change}, {target_high_change}, {target_low_change}, {targ

In [18]:
ctx[f'glob_{pop_stat_type}_peg_ratio'], ctx[f'sec_{pop_stat_type}_peg_ratio']

(95, 93)

In [19]:
ctx[f'sec_{pop_stat_type}_recommendation_mean']

50

In [28]:
feature_names = [
	'yfsym',
	'ibsym',
	'company_name',
	'country',
	# 'display_country',
	'sector',
	# 'display_sector',
	'industry',
	
	'price',
	'shares',

	'yield',
	f'sec_{pop_stat_type}_yield',
	f'glob_{pop_stat_type}_yield',

	'recommendation_key',
	'recommendation_mean',
	f'sec_{pop_stat_type}_recommendation_mean',
	f'glob_{pop_stat_type}_recommendation_mean',

	'peg_ratio',
	f'sec_{pop_stat_type}_peg_ratio',
	f'glob_{pop_stat_type}_peg_ratio',

	'change_52w',
	f'sec_{pop_stat_type}_change_52w',
	f'glob_{pop_stat_type}_change_52w',

	'perf52w',
	f'sec_{pop_stat_type}_perf52w',
	f'glob_{pop_stat_type}_perf52w',

	'market_cap',
	f'sec_{pop_stat_type}_market_cap',
	f'glob_{pop_stat_type}_market_cap',
	'market_cap_raw',
	
	'overall_risk',
	f'sec_{pop_stat_type}_overall_risk',
	f'glob_{pop_stat_type}_overall_risk',

	'held_percent_institutions',
	f'sec_{pop_stat_type}_held_percent_institutions',
	f'glob_{pop_stat_type}_held_percent_institutions',

	'held_percent_insiders',
	f'sec_{pop_stat_type}_held_percent_insiders',
	f'glob_{pop_stat_type}_held_percent_insiders',	
]

table = [[ctx[k] for k in feature_names] for ctx in pop]

df = pd.DataFrame(table, columns=feature_names)

In [29]:
# import dtale
# dtale.show(df)

In [30]:
# from pandasgui import show
# show(df)

In [31]:
csv_column_titles = {
	'yfsym': 'Yahoo Symbol',
	'ibsym': 'IBKR Symbol',
	'company_name': 'Company Name',
	'country': 'Country',
	'display_country': 'Country',
	'sector': 'Sector',
	'display_sector': 'Sector',
	'industry': 'Industry',
	
	'price': 'Price',
	'shares': 'Current Shares',
	
	'yield': 'Dividend Yield',
	f'sec_{pop_stat_type}_yield': 'Sector Percentile of Dividend Yield',
	f'glob_{pop_stat_type}_yield': 'Overall Percentile of Dividend Yield',
	
	'recommendation_key': 'Recommendation Key',
	'recommendation_mean': 'Recommendation Mean',
	f'sec_{pop_stat_type}_recommendation_mean': 'Sector Percentile of Recommendation Mean',
	f'glob_{pop_stat_type}_recommendation_mean': 'Overall Percentile of Recommendation Mean',
	
	'peg_ratio': 'PEG Ratio',
	f'sec_{pop_stat_type}_peg_ratio': 'Sector Percentile of PEG Ratio',
	f'glob_{pop_stat_type}_peg_ratio': 'Overall Percentile of PEG Ratio',
	
	'change_52w': 'Change 52w',
	f'sec_{pop_stat_type}_change_52w': 'Sector Percentile of Change 52w',
	f'glob_{pop_stat_type}_change_52w': 'Overall Percentile of Change 52w',
	
	'perf52w': '52 Week Relative Price (1=52 week high, 0=52 week low)',
	f'sec_{pop_stat_type}_perf52w': 'Sector Percentile of 52 Week Relative Price',
	f'glob_{pop_stat_type}_perf52w': 'Overall Percentile of 52 Week Relative Price',
	
	'market_cap': 'Market Cap',
	f'sec_{pop_stat_type}_market_cap': 'Sector Percentile of Market Cap',
	f'glob_{pop_stat_type}_market_cap': 'Overall Percentile of Market Cap',
	# 'market_cap_raw': 'Market Cap Raw',
	
	'overall_risk': 'Overall Risk (1-10)',
	f'sec_{pop_stat_type}_overall_risk': 'Sector Percentile of Overall Risk',
	f'glob_{pop_stat_type}_overall_risk': 'Overall Percentile of Overall Risk',
	
	'held_percent_institutions': 'Percent of Shares Held by Institutions',
	f'sec_{pop_stat_type}_held_percent_institutions': 'Sector Percentile of Percent of Shares Held by Institutions',
	f'glob_{pop_stat_type}_held_percent_institutions': 'Overall Percentile of Percent of Shares Held by Institutions',
	
	'held_percent_insiders': 'Percent of Shares Held by Insiders',
	f'sec_{pop_stat_type}_held_percent_insiders': 'Sector Percentile of Percent of Shares Held by Insiders',
	f'glob_{pop_stat_type}_held_percent_insiders': 'Overall Percentile of Percent of Shares Held by Insiders',
}
df.rename(columns=csv_column_titles).to_csv(misc.assets_root() / 'stocks.csv', index=False);